# DSCI 591 Capstone I
## Project: Decoding Crime in NYC
### Team: Safe City
#### By Min Sung Kim, Pranav Menon, Brian Kong, Josh Lister

# 1. Acquisition Code for NYC Crime Data

## 1.1 Data Acquisition Code for NYC Crime Data
1. In this project we will use NYC crime data for 5 years between 2016 and 2020.
2. Data is obtained through API provided by NYC Open Data in csv format.
    - Client key is needed for stable access.
3. Data can be obtained by specifying starting and ending date (1st and 2nd parameters).
4. Data can be obtained by specifying selected columns in a string (3rd parameter, default='' (all))
5. Data limit can be set (4th parameter, default = 1,000,000)
   - Default value is 1,000,000 and this is good for one whole year data length.
6. Data set contains 35 columns and the data length per year is around half million.

In [5]:
# make sure to install these packages before running:
import pandas as pd
from sodapy import Socrata

# parameter: start_date, end_date, columns_to_select, data_limit
# return: csv data
# Usage example
# start_date = '2018-01-01'
# end_date = '2018-12-31'
# columns_to_select: name of selected attributes in string separated by commas
#   default = '' (All attributes)
# result = get_ny_cmplnt(start_date, end_date)
# data_limit: Number of data lines per query
#   default = 1000000 (good for one whole year)
def get_ny_cmplnt(start_date, end_date, columns_to_select='', data_limit=1000000):
    
    # Access NY Open Data API
    client = Socrata("data.cityofnewyork.us","VjzUxyZ7GpGEy1jBXr5EzZpz3", 
                username = "mk3268@drexel.edu", password="Mact98xz!")

    # Make string data for where option in query
    start_str = "cmplnt_fr_dt >= '" + start_date + "'"
    end_str   = "cmplnt_fr_dt <= '" + end_date + "'"
    where_str = start_str +' and '+ end_str
    
    # Request and get result
    result = client.get("qgea-i56i", limit = data_limit, 
                         where = where_str, select = columns_to_select)
    
    # Convert to pandas DataFrame
    result_csv = pd.DataFrame.from_records(result)
    return(result_csv)

## 1.2 Selection of columns
- From the previous work, we identified the colunms that are not essential for the data analysis. 
- Compliant number('cmplnt_num') is not necesssary for the data analysis, but it is retained for book-keeping and further reference.

### Columns to drop
See the data description for detail.

- 'addr_pct_cd'
- 'cmplnt_to_dt'
- 'cmplnt_to_tm'
- 'rpt_dt'
- 'ky_cd'
- 'pd_cd'
- 'pd_desc'
- 'juris_desc'
- 'jurisdiction_code'
- 'x_coord_cd'
- 'y_coord_cd'
- 'lat_lon'
- 'housing_psa'
- 'hadevelopt'
- 'transit_district'
- 'patro_boro'

Note: We use 'boro_nm', 'latitude', and 'longitude'.

## 1.3 Get NYC crime data for 2016 ~ 2020

- Since the data length for 5 year record is aobut 2.5 million, it will take long time to get the data. 
- Therefore, it is better to obtain the data yearly and save them to csv files individually and merge them for the data processiong. 

In [6]:
# NYC historic compliant data for 5 years (2016~2020)
data_to_collect = [['2016-01-01', '2016-12-31', 'ny_crm_2016.csv'], 
                   ['2017-01-01', '2017-12-31', 'ny_crm_2017.csv'],
                   ['2018-01-01', '2018-12-31', 'ny_crm_2018.csv'],
                   ['2019-01-01', '2019-12-31', 'ny_crm_2019.csv'],
                   ['2020-01-01', '2020-12-31', 'ny_crm_2020.csv']]

columns_to_select = "cmplnt_num, cmplnt_fr_dt, cmplnt_fr_tm, ofns_desc, crm_atpt_cptd_cd,  \
    law_cat_cd, boro_nm, loc_of_occur_desc, prem_typ_desc, susp_age_group, susp_race, susp_sex, \
    latitude, longitude, vic_age_group, vic_race, vic_sex, parks_nm, station_name"

results_df = []
for data in data_to_collect:    
    start_date = data[0]
    end_date = data[1]
    csv_file = data[2]
    result_df = get_ny_cmplnt(start_date, end_date, columns_to_select)
    results_df.append(result_df)
    
    # Save individual data (one year) to a CSV file 
    result_df.to_csv(csv_file, index = False)  

In [7]:
# Print lengh and csv file name of data set per year
for i in range(5):
    print('Year = %d, rows = %d, csv_file = "%s"'%(2016+i, len(results_df[i]),data_to_collect[i][2]))

Year = 2016, rows = 478189, csv_file = "ny_crm_2016.csv"
Year = 2017, rows = 467809, csv_file = "ny_crm_2017.csv"
Year = 2018, rows = 461904, csv_file = "ny_crm_2018.csv"
Year = 2019, rows = 457543, csv_file = "ny_crm_2019.csv"
Year = 2020, rows = 404892, csv_file = "ny_crm_2020.csv"


## 1.4 Merging data for all 5 years

In [8]:
df_all = results_df[0].copy()
for i in range(1,5):
    df_all = df_all.append(results_df[i], ignore_index = True)

In [9]:
df_all

,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,prem_typ_desc,susp_age_group,susp_race,susp_sex,latitude,longitude,vic_age_group,vic_race,vic_sex,station_name,parks_nm
0,932184137,2016-01-01T00:00:00.000,00:00:00,SEX CRIMES,COMPLETED,FELONY,MANHATTAN,INSIDE,RESIDENCE - APT. HOUSE,45-64,WHITE HISPANIC,M,40.85148777400008,-73.93545796699993,<18,BLACK HISPANIC,F,NaN,NaN
1,524734897,2016-01-01T00:00:00.000,00:20:00,THEFT-FRAUD,COMPLETED,FELONY,BROOKLYN,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,40.64592203900003,-73.95096460799994,25-44,BLACK,F,NaN,NaN
2,425311840,2016-01-01T00:00:00.000,00:00:00,MISCELLANEOUS PENAL LAW,COMPLETED,FELONY,STATEN ISLAND,INSIDE,RESIDENCE - APT. HOUSE,25-44,WHITE HISPANIC,M,40.64472613100002,-74.07748315899995,UNKNOWN,UNKNOWN,E,NaN,NaN
3,211927249,2016-01-01T00:00:00.000,19:00:00,SEX CRIMES,COMPLETED,MISDEMEANOR,BRONX,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,40.85396027400003,-73.90012087499997,<18,BLACK HISPANIC,F,NaN,NaN
4,934187944,2016-01-01T00:00:00.000,12:00:00,SEX CRIMES,COMPLETED,MISDEMEANOR,QUEENS,INSIDE,RESIDENCE-HOUSE,UNKNOWN,UNKNOWN,U,40.72236368700004,-73.85147389399998,<18,WHITE HISPANIC,F,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270332,165691435,2020-12-31T00:00:00.000,20:40:00,DANGEROUS WEAPONS,COMPLETED,FELONY,BROOKLYN,NaN,STREET,NaN,NaN,NaN,40.679739379000075,-73.93739801899994,UNKNOWN,UNKNOWN,E,NaN,NaN
2270333,408014308,2020-12-31T00:00:00.000,15:00:00,PETIT LARCENY,COMPLETED,MISDEMEANOR,MANHATTAN,FRONT OF,RESIDENCE - PUBLIC HOUSING,UNKNOWN,UNKNOWN,U,40.72380251300007,-73.97906933499998,45-64,WHITE HISPANIC,M,NaN,NaN
2270334,152189917,2020-12-31T00:00:00.000,21:30:00,GRAND LARCENY,COMPLETED,FELONY,MANHATTAN,INSIDE,HOTEL/MOTEL,NaN,NaN,NaN,40.782076545000045,-73.98083480299994,25-44,WHITE HISPANIC,F,NaN,NaN
2270335,819066061,2020-12-31T00:00:00.000,20:22:00,DANGEROUS WEAPONS,COMPLETED,FELONY,BROOKLYN,NaN,STREET,NaN,NaN,NaN,40.66934487900005,-73.873801127,UNKNOWN,UNKNOWN,E,NaN,NaN


In [10]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404892 entries, 0 to 404891
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   cmplnt_num         404892 non-null  object
 1   cmplnt_fr_dt       404892 non-null  object
 2   cmplnt_fr_tm       404892 non-null  object
 3   ofns_desc          404887 non-null  object
 4   crm_atpt_cptd_cd   404892 non-null  object
 5   law_cat_cd         404892 non-null  object
 6   boro_nm            404429 non-null  object
 7   loc_of_occur_desc  339611 non-null  object
 8   prem_typ_desc      403788 non-null  object
 9   latitude           404892 non-null  object
 10  longitude          404892 non-null  object
 11  vic_age_group      404891 non-null  object
 12  vic_race           404891 non-null  object
 13  vic_sex            404891 non-null  object
 14  susp_age_group     312794 non-null  object
 15  susp_race          312794 non-null  object
 16  susp_sex           3

In [11]:
# Write to csv file
df_all.to_csv("ny_crm_all.csv", index = False) 
print('Year 2016 ~ 2020 NY crime data is written to "ny_crm_all.csv".')

Year 2016 ~ 2020 NY crime data is written to "ny_crm_all.csv".
